# Hyperopt Nonlinearities

#### Important Note: SQGL Learning Rates will be found using 3_hyperopt_LR_SQGL.ipynb

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from fit_params.fit_functions import *

import torch
import torch.nn as nn
from torch.nn.parameter import Parameter

from custompackage.load_architecture import *
from custompackage.load_data import *
from custompackage.traintestloop import *

import sparselinear as sl
import custompackage.sl_custom as slc

import hyperopt as hp
%matplotlib inline

np.set_printoptions(suppress=True)

import pickle
import time
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

## Hyperoptimization Class
- Activation functions: ReLU, LReLU, Sigmoid, SiLU, Swish, SQGL
- Learning-rate focused


In [2]:
class Hyperoptimizer():
    '''
    - Includes objective function methods
    - Includes run trials
    - Includes "while true" loop
    - Parameters will be: dataset, file location, types of hyperparameters optimized
    - Activation functions: ReLU, LReLU, Sigmoid, SiLU, Swish, SQGL
    - Learning-rate focused
    '''
    def __init__(self, selector=0, dataset='mnist', activation='sqgl', DIR='./results/temp/', 
                 threshold=300, Synapse=False, leak=0.01):
        self.selector = selector
        self.dataset = dataset
        self.activation = activation
        self.DIR = DIR
        self.threshold = threshold
        self.Synapse = Synapse
        self.leak = leak
        
        if self.Synapse == True:
            self.syn_key = 'syn'
        else:
            self.syn_key = 'nosyn'
        
        self.space = hp.choice('a',
                                [
                                    {
                                        'lr': hp.uniform('lr', 0.0001, 0.01)
                                    }
                                ])
    
    def objective(self, params):
        
        #Hyperparameter to be optimized
        lr = params['lr']
        
        #SQGL parameters if appropriate
        atten = 0.5
        scale = 1
        
        # Initialize settings
        bs = 256
        weighting = 'paired'
        trials = 10
        epochs = 2000
        trees_set = [1]
        trees = trees_set[0]

        # Load class-dataset list
        classes = np.load('./results/classes.npy', allow_pickle=True)
        classes = [classes[self.selector]]


        # For each dataset enumerated from classes list
        for j, (t1, t2, ds) in enumerate(classes):
            print(t1, t2, ds)
            # Load data loaders
            trainloaders, validloaders, testloader = dataset_weighted_split_all(bs, t1, t2, weighting, trials, ds, permute=False)
            # Initialize input size for model initialization purposes
            input_size = trainloaders[0].dataset.tensors[0][0].shape[0]
            # For each trial
            # Initialize the ktree model
            model = ktree_sparse(ds=ds, Repeats=trees, Padded=True, Activation=self.activation, learn=False, 
                                 alpha=1, beta=1, gamma=1, scale=scale, atten=atten,
                                 synapse=self.Synapse, leak=self.leak).cuda()
            #Train and test ktree, assigning loss and acc values
            loss_curve, acc_curve, loss, acc, model_t = train_test_ktree_sparse(model, trainloaders[0],
                                                                                validloaders[0], validloaders[0], 
                                                                                epochs = epochs, randorder=False,
                                                                                lr=lr)
            break
        if loss > 9:
            status = 'fail'
        else:
            status = 'ok'
        return {
            'loss': loss,
            'acc' : acc,
            'status': status,
            }
    
    def run_trials(self):

        trials_step = 1  # how many additional trials to do after loading saved trials. 1 = save after iteration
        max_trials = 1  # initial max_trials. put something small to not have to wait


        try:  # try to load an already saved trials object, and increase the max
            trials = pickle.load(open(self.DIR + 'ktree_'+self.activation+'_'+self.dataset+'_'+self.syn_key+'.hyperopt', "rb"))
            print("Found saved Trials! Loading...")
            max_trials = len(trials.trials) + trials_step
            print("Rerunning from {} trials to {} (+{}) trials".format(len(trials.trials), max_trials, trials_step))
        except:  # create a new trials object and start searching
            trials = Trials()

        best = fmin(fn=self.objective, space=self.space, algo=tpe.suggest, max_evals=max_trials, trials=trials)

        print("Best:", best)

        # save the trials object
        with open(self.DIR + 'ktree_'+self.activation+'_'+self.dataset+'_'+self.syn_key+'.hyperopt', "wb") as f:
            pickle.dump(trials, f)
            
        if max_trials >= self.threshold:
            return(False)
        else:
            return(True)
    
    def loop_to_threshold(self):
        run_status = True
        while run_status:
            run_status = self.run_trials()
        print('Threshold of '+str(self.threshold)+' reached')

## Hyperopt learning rate for each activation function for each dataset
ReLU, LReLU, Sigmoid, SiLU, Swish, SQGL

### ReLU nonlinearity

In [ ]:
# Initialize hyperoptimizer
hyperoptimizer_mnist = Hyperoptimizer(selector=0, 
                                      dataset='mnist',
                                      activation='relu',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False,
                                      leak=0)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_mnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_fmnist = Hyperoptimizer(selector=1, 
                                      dataset='fmnist',
                                      activation='relu',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False,
                                      leak=0)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_fmnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_kmnist = Hyperoptimizer(selector=2, 
                                      dataset='kmnist',
                                      activation='relu',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False,
                                      leak=0)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_kmnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_emnist = Hyperoptimizer(selector=3, 
                                      dataset='emnist',
                                      activation='relu',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False,
                                      leak=0)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_emnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_svhn = Hyperoptimizer(selector=4, 
                                      dataset='svhn',
                                      activation='relu',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False,
                                      leak=0)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_svhn.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_usps = Hyperoptimizer(selector=5, 
                                      dataset='usps',
                                      activation='relu',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False,
                                      leak=0)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_usps.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_cifar10 = Hyperoptimizer(selector=6, 
                                      dataset='cifar10',
                                      activation='relu',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False,
                                      leak=0)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_cifar10.loop_to_threshold()

### LReLU nonlinearity

In [ ]:
# Initialize hyperoptimizer
hyperoptimizer_mnist = Hyperoptimizer(selector=0, 
                                      dataset='mnist',
                                      activation='relu',,
                                      DIR='./results/20201227/l_', 
                                      threshold=300,
                                      Synapse=False,
                                      leak=0.01)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_mnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_fmnist = Hyperoptimizer(selector=1, 
                                      dataset='fmnist',
                                      activation='relu',
                                      DIR='./results/20201227/l_', 
                                      threshold=300,
                                      Synapse=False,
                                      leak=0.01)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_fmnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_kmnist = Hyperoptimizer(selector=2, 
                                      dataset='kmnist',
                                      activation='relu',
                                      DIR='./results/20201227/l_', 
                                      threshold=300,
                                      Synapse=False,
                                      leak=0.01)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_kmnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_emnist = Hyperoptimizer(selector=3, 
                                      dataset='emnist',
                                      activation='relu',,
                                      DIR='./results/20201227/l_', 
                                      threshold=300,
                                      Synapse=False,
                                      leak=0.01)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_emnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_svhn = Hyperoptimizer(selector=4, 
                                      dataset='svhn',
                                      activation='relu',
                                      DIR='./results/20201227/l_', 
                                      threshold=300,
                                      Synapse=False,
                                      leak=0.01)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_svhn.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_usps = Hyperoptimizer(selector=5, 
                                      dataset='usps',
                                      activation='relu',
                                      DIR='./results/20201227/l_', 
                                      threshold=300,
                                      Synapse=False,
                                      leak=0.01)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_usps.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_cifar10 = Hyperoptimizer(selector=6, 
                                      dataset='cifar10',
                                      activation='relu',
                                      DIR='./results/20201227/l_', 
                                      threshold=300,
                                      Synapse=False,
                                      leak=0.01)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_cifar10.loop_to_threshold()

### Sigmoid nonlinearity

In [ ]:
# Initialize hyperoptimizer
hyperoptimizer_mnist = Hyperoptimizer(selector=0, 
                                      dataset='mnist',
                                      activation='sigmoid',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_mnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_fmnist = Hyperoptimizer(selector=1, 
                                      dataset='fmnist',
                                      activation='sigmoid',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_fmnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_kmnist = Hyperoptimizer(selector=2, 
                                      dataset='kmnist',
                                      activation='sigmoid',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_kmnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_emnist = Hyperoptimizer(selector=3, 
                                      dataset='emnist',
                                      activation='sigmoid',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_emnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_svhn = Hyperoptimizer(selector=4, 
                                      dataset='svhn',
                                      activation='sigmoid',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_svhn.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_usps = Hyperoptimizer(selector=5, 
                                      dataset='usps',
                                      activation='sigmoid',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_usps.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_cifar10 = Hyperoptimizer(selector=6, 
                                      dataset='cifar10',
                                      activation='sigmoid',
                     ,                 DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_cifar10.loop_to_threshold()

### SiLU nonlinearity

In [ ]:
# Initialize hyperoptimizer
hyperoptimizer_mnist = Hyperoptimizer(selector=0, 
                                      dataset='mnist',
                                      activation='silu',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_mnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_fmnist = Hyperoptimizer(selector=1, 
                                      dataset='fmnist',
                                      activation='silu',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_fmnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_kmnist = Hyperoptimizer(selector=2, 
                                      dataset='kmnist',
                                      activation='silu',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_kmnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_emnist = Hyperoptimizer(selector=3, 
                                      dataset='emnist',
                                      activation='silu',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_emnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_svhn = Hyperoptimizer(selector=4, 
                                      dataset='svhn',
                                      activation='silu',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_svhn.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_usps = Hyperoptimizer(selector=5, 
                                      dataset='usps',
                                      activation='silu',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_usps.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_cifar10 = Hyperoptimizer(selector=6, 
                                      dataset='cifar10',
                                      activation='silu',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_cifar10.loop_to_threshold()

### Swish nonlinearity

In [ ]:
# Initialize hyperoptimizer
hyperoptimizer_mnist = Hyperoptimizer(selector=0, 
                                      dataset='mnist',
                                      activation='swish',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_mnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_fmnist = Hyperoptimizer(selector=1, 
                                      dataset='fmnist',
                                      activation='swish',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_fmnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_kmnist = Hyperoptimizer(selector=2, 
                                      dataset='kmnist',
                                      activation='swish',,
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_kmnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_emnist = Hyperoptimizer(selector=3, 
                                      dataset='emnist',
                                      activation='swish',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_emnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_svhn = Hyperoptimizer(selector=4, 
                                      dataset='svhn',
                                      activation='swish',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_svhn.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_usps = Hyperoptimizer(selector=5, 
                                      dataset='usps',
                                      activation='swish',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_usps.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_cifar10 = Hyperoptimizer(selector=6, 
                                      dataset='cifar10',
                                      activation='swish',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_cifar10.loop_to_threshold()

### SQGL nonlinearity without Synapse

In [ ]:
# Initialize hyperoptimizer
hyperoptimizer_mnist = Hyperoptimizer(selector=0, 
                                      dataset='mnist',
                                      activation='sqgl',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_mnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_fmnist = Hyperoptimizer(selector=1, 
                                      dataset='fmnist',
                                      activation='sqgl',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_fmnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_kmnist = Hyperoptimizer(selector=2, 
                                      dataset='kmnist',
                                      activation='sqgl',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_kmnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_emnist = Hyperoptimizer(selector=3, 
                                      dataset='emnist',
                                      activation='sqgl',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_emnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_svhn = Hyperoptimizer(selector=4, 
                                      dataset='svhn',
                                      activation='sqgl',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_svhn.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_usps = Hyperoptimizer(selector=5, 
                                      dataset='usps',
                                      activation='sqgl',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_usps.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_cifar10 = Hyperoptimizer(selector=6, 
                                      dataset='cifar10',
                                      activation='sqgl',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=False)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_cifar10.loop_to_threshold()

### SQGL nonlinearity with Synapse

In [ ]:
# Initialize hyperoptimizer
hyperoptimizer_mnist = Hyperoptimizer(selector=0, 
                                      dataset='mnist',
                                      activation='sqgl',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=True)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_mnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_fmnist = Hyperoptimizer(selector=1, 
                                      dataset='fmnist',
                                      activation='sqgl',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=True)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_fmnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_kmnist = Hyperoptimizer(selector=2, 
                                      dataset='kmnist',
                                      activation='sqgl',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=True)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_kmnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_emnist = Hyperoptimizer(selector=3, 
                                      dataset='emnist',
                                      activation='sqgl',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=True)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_emnist.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_svhn = Hyperoptimizer(selector=4, 
                                      dataset='svhn',
                                      activation='sqgl',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=True)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_svhn.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_usps = Hyperoptimizer(selector=5, 
                                      dataset='usps',
                                      activation='sqgl',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=True)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_usps.loop_to_threshold()


# Initialize hyperoptimizer
hyperoptimizer_cifar10 = Hyperoptimizer(selector=6, 
                                      dataset='cifar10',
                                      activation='sqgl',
                                      DIR='./results/20201227/', 
                                      threshold=300,
                                      Synapse=True)

# Run hyperoptimizer until threshold for number of trials is reached
hyperoptimizer_cifar10.loop_to_threshold()

## Collect best learning rates

In [18]:

# Get list of classes
classes = np.load('./results/classes.npy', allow_pickle=True)
class_names = classes[:,2]


activations = ['relu', 'lrelu','sigmoid','swish','sqgl']

syn_keys = ['syn', 'nosyn']

# Initialize dictionary
sqgl_dict = {}

# If synapse included, load from correct folder by assigning DIR path
DIR = './results/20201227/'
    
    
# Load best hyperparameters for each dataset
act_dict = {}
for act in activations:

    dataset_dict = {}
    for dataset in class_names:

        syn_dict = {}
        if act == 'sqgl':
            for syn_key in syn_keys:
                trials = pickle.load(open(DIR + 'ktree_'+act+'_'+dataset+'_'+syn_key+'.hyperopt', "rb"))
                lr = round(trials.best_trial['misc']['vals']['lr'][0], 4)
                syn_dict[syn_key] = {'lr' : lr}
        else:
            syn_key = 'nosyn'
            
            if act == 'lrelu':
                trials = pickle.load(open(DIR + 'l_ktree_relu_'+dataset+'_'+syn_key+'.hyperopt', "rb"))
                lr = round(trials.best_trial['misc']['vals']['lr'][0], 4)
                syn_dict[syn_key] = {'lr' : lr}
            else:
                trials = pickle.load(open(DIR + 'ktree_'+act+'_'+dataset+'_'+syn_key+'.hyperopt', "rb"))
                lr = round(trials.best_trial['misc']['vals']['lr'][0], 4)
                syn_dict[syn_key] = {'lr' : lr}

        dataset_dict[dataset] = syn_dict

    act_dict[act] = dataset_dict

for act in activations:
    print(act)
    for dataset in class_names:
        
        print(dataset, act_dict[act][dataset])
        


with open('./results/hyperparameters/ktree_lr.hyperopt', 'wb') as handle:
    pickle.dump(act_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

relu
mnist {'nosyn': {'lr': 0.0058}}
fmnist {'nosyn': {'lr': 0.006}}
kmnist {'nosyn': {'lr': 0.001}}
emnist {'nosyn': {'lr': 0.0043}}
svhn {'nosyn': {'lr': 0.0079}}
usps {'nosyn': {'lr': 0.0021}}
cifar10 {'nosyn': {'lr': 0.0024}}
lrelu
mnist {'nosyn': {'lr': 0.0059}}
fmnist {'nosyn': {'lr': 0.0006}}
kmnist {'nosyn': {'lr': 0.0051}}
emnist {'nosyn': {'lr': 0.0018}}
svhn {'nosyn': {'lr': 0.0081}}
usps {'nosyn': {'lr': 0.0056}}
cifar10 {'nosyn': {'lr': 0.004}}
sigmoid
mnist {'nosyn': {'lr': 0.0068}}
fmnist {'nosyn': {'lr': 0.0015}}
kmnist {'nosyn': {'lr': 0.0049}}
emnist {'nosyn': {'lr': 0.0077}}
svhn {'nosyn': {'lr': 0.0012}}
usps {'nosyn': {'lr': 0.0057}}
cifar10 {'nosyn': {'lr': 0.0083}}
swish
mnist {'nosyn': {'lr': 0.0079}}
fmnist {'nosyn': {'lr': 0.0001}}
kmnist {'nosyn': {'lr': 0.0099}}
emnist {'nosyn': {'lr': 0.0018}}
svhn {'nosyn': {'lr': 0.0003}}
usps {'nosyn': {'lr': 0.01}}
cifar10 {'nosyn': {'lr': 0.0002}}
sqgl
mnist {'syn': {'lr': 0.0007}, 'nosyn': {'lr': 0.003}}
fmnist {'syn'

## Hyperoptimize learning rate for FCNN

In [15]:
class Hyperoptimizer_fcnn():
    '''
    - Includes objective function methods
    - Includes run trials
    - Includes "while true" loop
    - Parameters will be: dataset, file location, types of hyperparameters optimized
    - Activation functions: ReLU, LReLU, Sigmoid, SiLU, Swish, SQGL
    - Learning-rate focused
    '''
    def __init__(self, selector=0, dataset='mnist', activation='sqgl', DIR='./results/temp/', 
                 threshold=300, Synapse=False, leak=0.01):
        self.selector = selector
        self.dataset = dataset
        self.activation = activation
        self.DIR = DIR
        self.threshold = threshold
        self.Synapse = Synapse
        self.leak = leak
        
        if self.Synapse == True:
            self.syn_key = 'syn'
        else:
            self.syn_key = 'nosyn'
        
        self.space = hp.choice('a',
                                [
                                    {
                                        'lr': hp.uniform('lr', 0.0001, 0.01)
                                    }
                                ])
    
    def objective(self, params):
        
        #Hyperparameter to be optimized
        lr = params['lr']
        
        #SQGL parameters if appropriate
        
        atten = 0.5
        scale = 1
        
        # Initialize settings
        bs = 256
        weighting = 'paired'
        trials = 10
        epochs = 2000
        trees_set = [1]
        trees = trees_set[0]

        # Load class-dataset list
        classes = np.load('./results/classes.npy', allow_pickle=True)
        classes = [classes[self.selector]]


        # For each dataset enumerated from classes list
        for j, (t1, t2, ds) in enumerate(classes):
            print(t1, t2, ds)
            # Load data loaders
            trainloaders, validloaders, testloader = dataset_weighted_split_all(bs, t1, t2, weighting, trials, ds, permute=False)
            # Initialize input size for model initialization purposes
            input_size = trainloaders[0].dataset.tensors[0][0].shape[0]
            # For each trial
            # Initialize the ktree model
            if self.Synapse == True:
                model = synapse_fcnn(Input_size=input_size, Hidden_size=trees*2, Output_size=1,
                                     Activation='sqgl', learn=False, alpha=1, beta=1, gamma=1, 
                                     scale=scale, atten=atten, leak=self.leak).cuda()
            else:
                model = simple_fcnn(Input_size=input_size, Hidden_size=trees*2, Output_size=1,
                                     Activation='sqgl', learn=False, alpha=1, beta=1, gamma=1, 
                                     scale=scale, atten=atten, leak=self.leak).cuda()
            #Train and test ktree, assigning loss and acc values
            loss_curve, acc_curve, loss, acc, model_t = train_test_ktree_sparse(model, trainloaders[0],
                                                                                validloaders[0], validloaders[0], 
                                                                                epochs = epochs, randorder=False,
                                                                                lr=lr)
            break
        if loss > 9:
            status = 'fail'
        else:
            status = 'ok'
        return {
            'loss': loss,
            'acc' : acc,
            'status': status,
            }
    
    def run_trials(self):

        trials_step = 1  # how many additional trials to do after loading saved trials. 1 = save after iteration
        max_trials = 1  # initial max_trials. put something small to not have to wait


        try:  # try to load an already saved trials object, and increase the max
            trials = pickle.load(open(self.DIR + 'fcnn_'+self.activation+'_'+self.dataset+'_'+self.syn_key+'.hyperopt', "rb"))
            print("Found saved Trials! Loading...")
            max_trials = len(trials.trials) + trials_step
            print("Rerunning from {} trials to {} (+{}) trials".format(len(trials.trials), max_trials, trials_step))
        except:  # create a new trials object and start searching
            trials = Trials()

        best = fmin(fn=self.objective, space=self.space, algo=tpe.suggest, max_evals=max_trials, trials=trials)

        print("Best:", best)

        # save the trials object
        with open(self.DIR + 'fcnn_'+self.activation+'_'+self.dataset+'_'+self.syn_key+'.hyperopt', "wb") as f:
            pickle.dump(trials, f)
            
        if max_trials >= self.threshold:
            return(False)
        else:
            return(True)
    
    def loop_to_threshold(self):
        run_status = True
        while run_status:
            run_status = self.run_trials()
        print('Threshold of '+str(self.threshold)+' reached')

### ReLU Nonlinearity

In [ ]:
# Initialize hyperoptimizer
classes = np.load('./results/classes.npy', allow_pickle=True)
classes = classes[:,2]

for dataset in classes:
    hyperoptimizer = Hyperoptimizer_fcnn(selector=0, 
                                          dataset=dataset,
                                          activation='relu',
                                          DIR='./results/20201229/', 
                                          threshold=300,
                                          Synapse=False,
                                          leak=0)

    # Run hyperoptimizer until threshold for number of trials is reached
    hyperoptimizer.loop_to_threshold()

### LReLU Nonlinearity

In [ ]:
# Initialize hyperoptimizer
classes = np.load('./results/classes.npy', allow_pickle=True)
classes = classes[:,2]

for dataset in classes:
    hyperoptimizer = Hyperoptimizer_fcnn(selector=0, 
                                          dataset=dataset,
                                          activation='relu',
                                          DIR='./results/20201229/l_', 
                                          threshold=300,
                                          Synapse=False,
                                          leak=0.01)

    # Run hyperoptimizer until threshold for number of trials is reached
    hyperoptimizer.loop_to_threshold()

### Sigmoid Nonlinearity

In [ ]:
# Initialize hyperoptimizer
classes = np.load('./results/classes.npy', allow_pickle=True)
classes = classes[:,2]

for dataset in classes:
    hyperoptimizer = Hyperoptimizer_fcnn(selector=0, 
                                          dataset=dataset,
                                          activation='sigmoid',
                                          DIR='./results/20201229/', 
                                          threshold=300,
                                          Synapse=False)

    # Run hyperoptimizer until threshold for number of trials is reached
    hyperoptimizer.loop_to_threshold()

### Swish Nonlinearity

In [ ]:
# Initialize hyperoptimizer
classes = np.load('./results/classes.npy', allow_pickle=True)
classes = classes[:,2]

for dataset in classes:
    hyperoptimizer = Hyperoptimizer_fcnn(selector=0, 
                                          dataset=dataset,
                                          activation='swish',
                                          DIR='./results/20201229/', 
                                          threshold=300,
                                          Synapse=False)

    # Run hyperoptimizer until threshold for number of trials is reached
    hyperoptimizer.loop_to_threshold()

### SQGL w/o Synapse Nonlinearity

In [ ]:
# Initialize hyperoptimizer
classes = np.load('./results/classes.npy', allow_pickle=True)
classes = classes[:,2]

for dataset in classes:
    hyperoptimizer = Hyperoptimizer_fcnn(selector=0, 
                                          dataset=dataset,
                                          activation='sqgl',
                                          DIR='./results/20201229/', 
                                          threshold=300,
                                          Synapse=False)

    # Run hyperoptimizer until threshold for number of trials is reached
    hyperoptimizer.loop_to_threshold()

### SQGL w/ Synapse Nonlinearity

In [ ]:
# Initialize hyperoptimizer
classes = np.load('./results/classes.npy', allow_pickle=True)
classes = classes[:,2]

for dataset in classes:
    hyperoptimizer = Hyperoptimizer_fcnn(selector=0, 
                                          dataset=dataset,
                                          activation='sqgl',
                                          DIR='./results/20201229/', 
                                          threshold=300,
                                          Synapse=True)

    # Run hyperoptimizer until threshold for number of trials is reached
    hyperoptimizer.loop_to_threshold()

## Collect learning rates for FCNN

In [19]:
# Get list of classes
classes = np.load('./results/classes.npy', allow_pickle=True)
class_names = classes[:,2]


activations = ['relu', 'lrelu','sigmoid','swish','sqgl']

syn_keys = ['syn', 'nosyn']

# Initialize dictionary
sqgl_dict = {}

# If synapse included, load from correct folder by assigning DIR path
DIR = './results/20201229/'
    
    
# Load best hyperparameters for each dataset
act_dict = {}
for act in activations:

    dataset_dict = {}
    for dataset in class_names:

        syn_dict = {}
        if act == 'sqgl':
            for syn_key in syn_keys:
                trials = pickle.load(open(DIR + 'fcnn_'+act+'_'+dataset+'_'+syn_key+'.hyperopt', "rb"))
                lr = round(trials.best_trial['misc']['vals']['lr'][0], 4)
                syn_dict[syn_key] = {'lr' : lr}
        else:
            syn_key = 'nosyn'
            
            if act == 'lrelu':
                trials = pickle.load(open(DIR + 'l_fcnn_relu_'+dataset+'_'+syn_key+'.hyperopt', "rb"))
                lr = round(trials.best_trial['misc']['vals']['lr'][0], 4)
                syn_dict[syn_key] = {'lr' : lr}
            else:
                trials = pickle.load(open(DIR + 'fcnn_'+act+'_'+dataset+'_'+syn_key+'.hyperopt', "rb"))
                lr = round(trials.best_trial['misc']['vals']['lr'][0], 4)
                syn_dict[syn_key] = {'lr' : lr}

        dataset_dict[dataset] = syn_dict

    act_dict[act] = dataset_dict

for act in activations:
    print(act)
    for dataset in class_names:
        
        print(dataset, act_dict[act][dataset])
        
# with open('./results/hyperparameters/fcnn_lr.hyperopt', 'wb') as handle:
#     pickle.dump(act_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

relu
mnist {'nosyn': {'lr': 0.0011}}
fmnist {'nosyn': {'lr': 0.0011}}
kmnist {'nosyn': {'lr': 0.001}}
emnist {'nosyn': {'lr': 0.0051}}
svhn {'nosyn': {'lr': 0.001}}
usps {'nosyn': {'lr': 0.0055}}
cifar10 {'nosyn': {'lr': 0.0059}}
lrelu
mnist {'nosyn': {'lr': 0.0045}}
fmnist {'nosyn': {'lr': 0.008}}
kmnist {'nosyn': {'lr': 0.0052}}
emnist {'nosyn': {'lr': 0.0095}}
svhn {'nosyn': {'lr': 0.0073}}
usps {'nosyn': {'lr': 0.0033}}
cifar10 {'nosyn': {'lr': 0.0082}}
sigmoid
mnist {'nosyn': {'lr': 0.0008}}
fmnist {'nosyn': {'lr': 0.0039}}
kmnist {'nosyn': {'lr': 0.0069}}
emnist {'nosyn': {'lr': 0.0047}}
svhn {'nosyn': {'lr': 0.0067}}
usps {'nosyn': {'lr': 0.0082}}
cifar10 {'nosyn': {'lr': 0.0059}}
swish
mnist {'nosyn': {'lr': 0.0081}}
fmnist {'nosyn': {'lr': 0.0028}}
kmnist {'nosyn': {'lr': 0.0032}}
emnist {'nosyn': {'lr': 0.0004}}
svhn {'nosyn': {'lr': 0.0001}}
usps {'nosyn': {'lr': 0.0036}}
cifar10 {'nosyn': {'lr': 0.003}}
sqgl
mnist {'syn': {'lr': 0.0063}, 'nosyn': {'lr': 0.0016}}
fmnist {'sy